In [1]:
import numpy as np
import os
import cv2 as cv

1. Выполните сохранение монохромного изображения в виде текстового файла.

In [2]:
img = cv.imread("image.jpg", cv.IMREAD_GRAYSCALE)
np.savetxt("original.txt", img, fmt="%d")

2. Реализуйте алгоритм вейвлет-преобразования Хаара для изображения.

In [3]:
def haar_transform_2d(image):
    img = image.astype(float)
    rows, cols = img.shape

    rows_eff = rows - rows % 2
    cols_eff = cols - cols % 2
    img = img[:rows_eff, :cols_eff]

    for i in range(rows_eff):
        even = img[i, :cols_eff:2]
        odd = img[i, 1:cols_eff:2]
        avg = (even + odd) / 2
        diff = (even - odd) / 2
        img[i, : cols_eff // 2] = avg
        img[i, cols_eff // 2 : cols_eff] = diff

    for j in range(cols_eff):
        even = img[:rows_eff:2, j]
        odd = img[1:rows_eff:2, j]
        avg = (even + odd) / 2
        diff = (even - odd) / 2
        img[: rows_eff // 2, j] = avg
        img[rows_eff // 2 : rows_eff, j] = diff

    return img

In [4]:
transformed = haar_transform_2d(img.copy())
rows, cols = transformed.shape
LL = transformed[: rows // 2, : cols // 2]
LH = transformed[: rows // 2, cols // 2 :]
HL = transformed[rows // 2 :, : cols // 2]
HH = transformed[rows // 2 :, cols // 2 :]

3. Выполните квантование высокочастотных компонент

In [5]:
def quantize(component, levels=4):
    min_val, max_val = component.min(), component.max()
    if max_val == min_val:
        return np.zeros_like(component, dtype=int)
    bins = np.linspace(min_val, max_val, levels + 1)
    quantized = np.digitize(component, bins) - 1
    quantized = np.clip(quantized, 0, levels - 1)
    return quantized

In [6]:
LH_q, HL_q, HH_q = quantize(LH), quantize(HL), quantize(HH)

 4. Сохраните получившийся массив значений  в текстовый файл

In [7]:
def rle_encode(arr):
    flat = arr.ravel()
    change_idx = np.where(flat[1:] != flat[:-1])[0] + 1
    splits = np.split(flat, change_idx)
    res = np.array([(s[0], len(s)) for s in splits], dtype=int)
    return res

order = [(LH_q, "LH"), (HL_q, "HL"), (HH_q, "HH")]

with open("haar.txt", "w") as f:

  # LL
  f.write(f"LL {LL.shape[0]} {LL.shape[1]}\n")
  np.savetxt(f, LL, fmt="%d")

  # LH, HL, HH
  for arr, name in order:
    encoded = rle_encode(arr)
    f.write(f"{name} {len(encoded)}\n")
    for val, count in encoded:
      f.write(f"{val} {count}\n")

In [8]:
original_size = os.path.getsize("original.txt")
compressed_size = os.path.getsize("haar.txt")
print("Оригинальный размер:", original_size, "bytes")
print("Размер после сжатия:", compressed_size, "bytes")
print(f"коэффицент сжатия: {original_size / compressed_size:.2f}")

Оригинальный размер: 2914297 bytes
Размер после сжатия: 868359 bytes
коэффицент сжатия: 3.36
